In [1]:
%load_ext autoreload
%autoreload 2

import pathlib 
import json
import imageio 
import numpy as np 
from tqdm import tqdm
import pandas as pd 

# Formatting Something-Something-v1

In [5]:
ss_files = pathlib.Path.home()/'Documents/gradschool/thesis/data/something-something/'

labels = pd.read_csv(ss_files/'something-something-v1-labels.csv', sep='\n', header=None)
labels = list(labels[0])
train = pd.read_csv(ss_files/'something-something-v1-train.csv', delimiter=';', sep='\n', header=None)
val = pd.read_csv(ss_files/'something-something-v1-validation.csv', delimiter=';', sep='\n', header=None)

train.columns = ['FileID', 'Labels']
val.columns = ['FileID', 'Labels']
print('Some Labels: ', labels[:4])
print('\nTraining Datafile:')
train.head()

Some Labels:  ['Holding something', 'Turning something upside down', 'Turning the camera left while filming something', 'Stacking number of something']

Training Datafile:


,FileID,Labels
0,100218,Something falling like a feather or paper
1,48032,Pretending to put something into something
2,10433,Letting something roll down a slanted surface
3,9426,Lifting something with something on it
4,62548,Showing something next to something


In [6]:
datafile_path = pathlib.Path.home()/'Documents/gradschool/thesis/data/tfrecords'
datafile_path.mkdir(parents=True, exist_ok=True)

for df, name in zip([train, val], ['train.txt', 'val.txt']):
    # compute the index of each label 
    df['label_idx'] = [labels.index(v) for v in df['Labels'].values]

    # write to train.txt or val.txt
    with open(str(datafile_path/name), 'w') as f: 
        for _, row in tqdm(df.iterrows()): 
            line = '{} {}\n'.format(row['FileID'], row['label_idx'])
            f.write(line)

86017it [00:07, 11956.51it/s]
11522it [00:00, 11886.23it/s]


In [51]:
# lets look at an example line in the file 
with open(str(datafile_path/'train.txt'), 'r') as f: 
    lines = f.readlines()
print('Example in file (fileID, label): ', lines[0])

Example in file (fileID, label):  100218 31



In [7]:
# how we access fileID (when the path in train.txt is not absolute)
datafile_prefix = pathlib.Path.home()/'Documents/gradschool/thesis/data'\
        '/something-something/20bn-something-something-v1'

# to make sure the paths are correct -- lets access an example:
example_fileid = lines[np.random.randint(len(lines))].split(' ')[0]
# format using datafile_prefix
example_path = datafile_prefix/'{}'.format(example_fileid)
print(example_path)

In [55]:
# success! 
list(example_path.iterdir())[:5]

[PosixPath('/Users/brennangebotys/Documents/gradschool/thesis/data/something-something/20bn-something-something-v1/59804/00046.jpg'),
 PosixPath('/Users/brennangebotys/Documents/gradschool/thesis/data/something-something/20bn-something-something-v1/59804/00052.jpg'),
 PosixPath('/Users/brennangebotys/Documents/gradschool/thesis/data/something-something/20bn-something-something-v1/59804/00053.jpg'),
 PosixPath('/Users/brennangebotys/Documents/gradschool/thesis/data/something-something/20bn-something-something-v1/59804/00047.jpg'),
 PosixPath('/Users/brennangebotys/Documents/gradschool/thesis/data/something-something/20bn-something-something-v1/59804/00051.jpg')]

# Writing TFRecords

In [9]:
from tfrecords import TFRecords4Video
# we will save the files in the same place as our datafiles
tfrecords_save_path = datafile_path
# datafile_file and datafile_prefix were already created and checked 
tfrv = TFRecords4Video(tfrecords_save_path, datafile_path, datafile_prefix, 'images')

## run this line to create all the tf records for each split 
# tfrv.create_tfrecords()

# Loading TFRecords

In [12]:
# get a str(path) to every training shard 
train_path = tfrecords_save_path/'train/'
train_shards = [str(path) for path in train_path.iterdir() if '.tfrecord' == path.suffix]
train_shards

['/Users/brennangebotys/Documents/gradschool/thesis/data/tfrecords/train/shard0.tfrecord']

In [25]:
import tensorflow.compat.v1 as tf 
from tfrecords import parse_example

# create dataset with train shards
# customize adding: .map(your_tf_function) after '.map(parse_example)\'
# Note: sharding allows us to shuffle the data efficiently
dataset = tf.data.TFRecordDataset(train_shards)\
    .map(parse_example)\
    .shuffle(100)\
    .batch(1)
# Note: for batchsize > 1 you'll have to resize the images all to the same size 
# i.e .map(your_resize_function)

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess: 
    video_data, labels, filenames = sess.run(next_element)
    
    print('Example Data:')
    print(video_data.shape, labels, filenames)

Example Data:
(1, 36, 100, 132, 3) [52] [b'/Users/brennangebotys/Documents/gradschool/thesis/data/something-something/20bn-something-something-v1/27684']
